**Cohesion** - It is degree to which elements of a certain class or function belong together. A class or function with strong cohesion has clear a responsibility, i.e. it has only one task, e.g. `math.cos`. It is easier to maintain and reuse class or function with strong cohesion

**Coupling** - It is a measure of how dependent two parts of code are on each other. High coupling is problematic because changing something in one part of a program means you need to change things in multiple places. There will always coupling in code because various parts of your code need to work together but the more coupling you introduce the more difficult it becomes to maintain your code. Solutions:
- Pass along data you need instead of object
- Move the code inside the object class

In [3]:
import string
import random

### Before

In [4]:
class VehicleRegistry:

    def generate_vehicle_id(self, length):
        return ''.join(random.choices(string.ascii_uppercase, k=length))

    def generate_vehicle_license(self, id):
        return f"{id[:2]}-{''.join(random.choices(string.digits, k=2))}-{''.join(random.choices(string.ascii_uppercase, k=2))}"


class Application:

    def register_vehicle(self, brand: string):
        """
        This method is doing a lot of different things. The method has a lot
        of responsibilites which means that this method has low cohesion. Also,
        this method has high coupling because it is directly relying on details
        of VehicleRegistry class (generate id and pass it to generate license),
        and if methods in VehicleRegistry class are modified then we will need 
        to modify this method too. Another problem - It is difficult to add another
        brand of vehicle. Data is also not stored logically - there is direct 
        coupling between brand and catalogue prices, tax percent depends on if 
        brand is electric or not. We have brand information and vehicle instance
        information -> separate into classes
        """
        # create a registry instance
        registry = VehicleRegistry()

        # generate a vehicle id of length 12
        vehicle_id = registry.generate_vehicle_id(12)

        # now generate a license plate for the vehicle
        # using the first two characters of the vehicle id
        license_plate = registry.generate_vehicle_license(vehicle_id)

        # compute the catalogue price
        catalogue_price = 0
        if brand == "Tesla Model 3":
            catalogue_price = 60000
        elif brand == "Volkswagen ID3":
            catalogue_price = 35000
        elif brand == "BMW 5":
            catalogue_price = 45000

        # compute the tax percentage (default 5% of the catalogue price, except for electric cars where it is 2%)
        tax_percentage = 0.05
        if brand == "Tesla Model 3" or brand == "Volkswagen ID3":
            tax_percentage = 0.02

        # compute the payable tax
        payable_tax = tax_percentage * catalogue_price

        # print out the vehicle registration information
        print("Registration complete. Vehicle information:")
        print(f"Brand: {brand}")
        print(f"Id: {vehicle_id}")
        print(f"License plate: {license_plate}")
        print(f"Payable tax: {payable_tax}")

app = Application()
app.register_vehicle("Volkswagen ID3")

Registration complete. Vehicle information:
Brand: Volkswagen ID3
Id: VOVSPKALSMSD
License plate: VO-05-RG
Payable tax: 700.0


### After

Identify where information is stored and how it is accessed. When you have defined the logical structure of information you can group the code around that. This results in lower coupling because the code is closer to the information it uses

In [5]:
class VehicleInformation:
    brand: str
    catalog_price: int
    electric: bool
        
    def __init__(self, brand, electric, catalog_price):
        self.brand = brand
        self.electric = electric
        self.catalog_price = catalog_price
        
    def calculate_tax(self):
        tax_percentage = 0.05
        if self.electric:
            tax_percentage = 0.02
        return tax_percentage * self.catalog_price
    
    def print(self):
        print(f"Brand: {self.brand}")
        print(f"Payable tax: {self.calculate_tax()}")
            

class Vehicle:
    id: str
    license_plate: str
    information: VehicleInformation
        
    def __init__(self, id, license_plate, information):
        self.id = id
        self.license_plate = license_plate
        self.information = information
        
    def print(self):
        print(f"ID: {self.id}")
        print(f"License plate: {self.license_plate}")
        self.information.print()
        
        
class VehicleRegistry:
    
    vehicle_information = {}  # Most likely a database
    
    def add_vehicle_information(self, brand, electric, catalog_price):
        self.vehicle_information[brand] = VehicleInformation(brand, electric, catalog_price)
        
    def __init__(self):
        self.add_vehicle_information("Tesla Model 3", True, 60000)
        self.add_vehicle_information("Volkswagen ID3", True, 53000)
        self.add_vehicle_information("BMW 5", False, 45000)

    def generate_vehicle_id(self, length):
        return ''.join(random.choices(string.ascii_uppercase, k=length))

    def generate_vehicle_license(self, id):
        return f"{id[:2]}-{''.join(random.choices(string.digits, k=2))}-{''.join(random.choices(string.ascii_uppercase, k=2))}"
    
    def create_vehicle(self, brand):
        # Generate a vehicle id of length 12
        vehicle_id = self.generate_vehicle_id(12)
        
        # Generate a license plate for the vehicle using the first two characters of the vehicle id
        license_plate = self.generate_vehicle_license(vehicle_id)
        
        return Vehicle(vehicle_id, license_plate, self.vehicle_information[brand])
        
        
class Application:

    def register_vehicle(self, brand: string):
        # Create a registry instance
        registry = VehicleRegistry()
        
        # Create vehicle
        vehicle = registry.create_vehicle(brand)
        
        # Print vehicle data
        vehicle.print()

app = Application()
app.register_vehicle("Volkswagen ID3")

ID: PBYEIVVNGUSS
License plate: PB-12-GC
Brand: Volkswagen ID3
Payable tax: 1060.0
